In [1]:
import datasets
from datasets import load_dataset
import openai
import os
import dspy
from dotenv import load_dotenv

load_dotenv()
root_path="."
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(root_path, 'cache')

openai_key = os.getenv('OPENAI_API_KEY')  # or replace with your API key (optional)

colbert_server = 'http://index.contextual.ai:8893/api/search'
# colbert_server = "http://20.102.90.50:2017/wiki17_abstracts"

lm = dspy.OpenAI(
    deployment_id=os.getenv('OPENAI_DEPLOYMENT'),
    api_key=os.getenv('OPENAI_API_KEY'),
    api_base=os.getenv('OPENAI_ENDPOINT'),
    api_provider= "azure",
    model_type="chat",
    api_version=os.getenv('OPENAI_API_VERSION'))
    # model='gpt-3.5-turbo', api_key=openai_key)

rm = dspy.ColBERTv2(url=colbert_server)

dspy.settings.configure(lm=lm, rm=rm)

squad = load_dataset("squad")

def get_squad_split(squad, split="validation"):
    """
    Use `split='train'` for the train split.

    Returns
    -------
    list of dspy.Example with attributes question, answer

    """
    data = zip(*[squad[split][field] for field in squad[split].features])
    exs = [dspy.Example(question=q, answer=a['text'][0]).with_inputs("question")
           for eid, title, context, q, a in data]
    return exs

squad_train = get_squad_split(squad, split="train")

squad_dev = get_squad_split(squad)

import random

random.seed(1)

dev_exs = random.sample(squad_dev, k=200)

from dspy.evaluate import answer_exact_match
from dspy.evaluate.evaluate import Evaluate

dev_evaluater = Evaluate(
    devset=dev_exs, # 200 examples
    num_threads=1,
    display_progress=True,
    display_table=5)

tiny_evaluater = Evaluate(
    devset=dev_exs[: 15],
    num_threads=1,
    display_progress=True,
    display_table=5)

In [2]:
from dsp.utils import deduplicate

class ContextQASignature(dspy.Signature):
    __doc__ = """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class OptimizedSignature(dspy.Signature):
    __doc__ = """The language model should understand the purpose of the question and provide a helpful and concise response that directly answers the question. It should use specific facts, information, and examples."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words", prefix="Response:")

class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

class SummarizeSignature(dspy.Signature):
    __doc__ = """Write a Brief summary of the given context. Include the information from the context which can answer the provided question."""

    context = dspy.InputField(desc="List of passages to summarize")
    question = dspy.InputField(desc="Question to be answered")
    summary = dspy.OutputField(desc="The summary of the above passages. 1-2 sentences.")

def validate_context_and_answer(example, pred, trace=None):
    return answer_exact_match(example=example, pred=pred, trace=trace) # and dspy.evaluate.answer_passage_match(example, pred)

from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, SignatureOptimizer

def bootstrap_random_search_optimize(model):
    config = dict(max_bootstrapped_demos=3, max_labeled_demos=3, num_threads=1)
    optimizer = BootstrapFewShotWithRandomSearch(metric=validate_context_and_answer, **config)
    return optimizer.compile(model, trainset=squad_train[:50], valset=dev_exs[:15])

def bootstrap_optimize(model):
    config = dict(max_bootstrapped_demos=3, max_labeled_demos=3)
    optimizer = BootstrapFewShot(metric=validate_context_and_answer, **config)
    return optimizer.compile(model, trainset=squad_train[:50], valset=dev_exs[:15])

def signature_optimizer(model):
    optimizer = SignatureOptimizer(breadth=3, depth=3, metric=validate_context_and_answer, verbose=True)
    kwargs = dict(num_threads=1, display_progress=True, display_table=False)
    return optimizer.compile(model, devset=dev_exs[:30], eval_kwargs=kwargs)

class RAGMultiHop(dspy.Module):
    def __init__(self, num_passages=3, hops=4):
        super().__init__()

        self.generate_queries = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(hops)]
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(OptimizedSignature)
        self.summarize = dspy.Predict(SummarizeSignature)
        self.hops = hops

    def forward(self, question):
        context = []
        
        for hop in range(self.hops):
            query = self.generate_queries[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context+passages)


        summary = self.summarize(context=context, question=question).summary
        
        prediction = self.generate_answer(context=summary, question=question)
        return dspy.Prediction(context=context, rationale=prediction.rationale, answer=prediction.answer)

# class RAG(dspy.Module):
#     def __init__(self, num_passages=3):
#         super().__init__()
#         self.retrieve = dspy.Retrieve(k=num_passages)
        
#         self.generate_answer = dspy.ChainOfThought(OptimizedSignature)

#     def forward(self, question):
#         context = self.retrieve(question).passages
        
#         prediction = self.generate_answer(context=context, question=question)
#         return dspy.Prediction(context=context, rationale=prediction.rationale, answer=prediction.answer)
    

# class CoT(dspy.Module):
#     def __init__(self):
#         super().__init__()
#         self.prog = dspy.ChainOfThought("question -> answer")
    
#     def forward(self, question):
#         return self.prog(question=question)

./cache/compiler


In [3]:
model = RAGMultiHop()
# model = signature_optimizer(model)
bootstrap_model = bootstrap_optimize(model)
# bootstrap_random_search_model = bootstrap_random_search_optimize(model)
# context = model.retrieve("What are some foods?").passages

# pred = model("What are some foods?")
# print(pred)
lm.inspect_history(n=1)

print("====================BASE MODEL====================")
tiny_evaluater(model, metric=answer_exact_match)
print("====================Bootstrap MODEL====================")
tiny_evaluater(bootstrap_model, metric=answer_exact_match)
# print("====================Bootstrap Random search MODEL====================")
# tiny_evaluater(bootstrap_random_search_model, metric=answer_exact_match)

 18%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 9/50 [00:32<02:29,  3.65s/it]


Bootstrapped 3 full traces after 10 examples in round 0.




The language model should understand the purpose of the question and provide a helpful and concise response that directly answers the question. It should use specific facts, information, and examples.

---

Question: What individuals live at Fatima House at Notre Dame?
Response: Retired priests and brothers

Question: To whom was John B. Kroc married?
Response: Ray Kroc

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Response: often between 1 and 5 words

---

Context: There are three student newspapers at Notre Dame.

Question: How many student news papers are found at Notre Dame?

Reasoning: Let's think step by step in order to produce the answer. We know that there are three student newspapers at Notre Dame.

Response: Three



====================BASE MODEL====================


Average Metric: 2 / 15  (13.3): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [01:01<00:00,  4.08s/it]


Average Metric: 2 / 15  (13.3%)


,question,example_answer,context,rationale,pred_answer,answer_exact_match
0,In 1517 who was Luther's bishop?,Albert of Mainz,"['Martin Luther | Martin Luther Martin Luther, (; ; 10 November 1483 – 18 February 1546) was a German professor of theology, composer, priest, monk,...","produce the answer. We know that in 1517, Martin Luther was a monk and theologian in Germany. As a monk, Luther was under the authority...",Albrecht von Brandenburg,False
1,When was the construction that changed the Rhine's Delta?,20th Century,['Delta Works | Delta Works The Delta Works () is a series of construction projects in the southwest of the Netherlands to protect a...,produce the answer. We know that the construction that changed the Rhine's Delta took place in the second half of the 20th century.,Second half of the 20th century.,False
2,How many companies were registered in Warsaw in 2006?,"304,016","['Warsaw | such as Sydney, Istanbul, Amsterdam or Seoul. Warsaw, especially its city centre (""Śródmieście""), is home not only to many national institutions and government...","produce the answer. We know that in 2006, 304,016 companies were registered in Warsaw.","304,016",✔️ [True]
3,What is the CJEU's duty?,"to ""ensure that in the interpretation and application of the Treaties the law is observed""",['European Union law | elected by the judges for three years. While TEU article 19(3) says the Court of Justice is the ultimate court to...,"produce the answer. We know that the CJEU's duty is to ""ensure that in the interpretation and application of the Treaties the law is observed.""...",Ensure law is observed.,False
4,What would a teacher do for someone who is cocky?,deflate,"['Reading comprehension | of strategies used to teach reading. Strategies vary according to the challenges like new concepts, unfamiliar vocabulary, long and complex sentences, etc....","produce the answer. We know that a teacher's role is to deflate the cocky and broaden the viewpoint of all students. Therefore, if someone is...",Address cockiness and promote humility.,False


...

Average Metric: 3 / 15  (20.0): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [01:08<00:00,  4.59s/it]

Average Metric: 3 / 15  (20.0%)



/anaconda/envs/cs224u-cpu/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


,question,example_answer,context,rationale,pred_answer,answer_exact_match
0,In 1517 who was Luther's bishop?,Albert of Mainz,"['Martin Luther | Martin Luther Martin Luther, (; ; 10 November 1483 – 18 February 1546) was a German professor of theology, composer, priest, monk,...",produce the answer. We know that Luther's bishop in 1517 was Albrecht von Brandenburg.,Albrecht von Brandenburg,False
1,When was the construction that changed the Rhine's Delta?,20th Century,"['Rhine | rivers and streams. Many rivers have been closed (""dammed"") and now serve as drainage channels for the numerous polders. The construction of Delta...",produce the answer. We know that the construction that changed the Rhine's Delta occurred in the second half of the 20th Century.,Second half of the 20th Century,False
2,How many companies were registered in Warsaw in 2006?,"304,016","['Warsaw | such as Sydney, Istanbul, Amsterdam or Seoul. Warsaw, especially its city centre (""Śródmieście""), is home not only to many national institutions and government...","produce the answer. We know that in 2006, 304,016 companies were registered in Warsaw.","304,016",✔️ [True]
3,What is the CJEU's duty?,"to ""ensure that in the interpretation and application of the Treaties the law is observed""",['European Union law | elected by the judges for three years. While TEU article 19(3) says the Court of Justice is the ultimate court to...,"produce the answer. We know that the CJEU's duty is to ""ensure that in the interpretation and application of the Treaties the law is observed""...",To ensure law is observed and oversee interpretation of EU law.,False
4,What would a teacher do for someone who is cocky?,deflate,"['Teacher | for the individual students accordingly. For example, an experienced teacher and parent described the place of a teacher in learning as follows: ""The...","produce the answer. We know that a teacher would ""deflate"" someone who is cocky.",Deflate,✔️ [True]


20.0

In [6]:
bootstrap_model("what are aborigines?")

Prediction(
    context=['Aborigines (mythology) | Aborigines (mythology) The Aborigines in Roman mythology are the oldest inhabitants of central Italy, connected in legendary history with Aeneas, Latinus and Evander. They were supposed to have descended from their mountain home near Reate (an ancient Sabine town) upon Latium, where they expelled the Siceli and subsequently settled down as Latini under a King Latinus. The most generally accepted etymology of the Latin word aborigines is that it derives from "ab origine", according to which they were the original inhabitants of the country, although Cato regarded them as Hellenic immigrants, not as a native Italian people. Other etymological explanations', 'Aborigines (mythology) | suggested are "arborigines", meaning "tree-born," and "aberrigines", meaning "nomads". Lycophron calls a people of central Italy, "Boreigonoi". The Aborigines were believed to have been descendants of the Pelasgians, the mythical ancestors of all Greek people

In [7]:
lm.inspect_history(n=1)





The language model should understand the purpose of the question and provide a helpful and concise response that directly answers the question. It should use specific facts, information, and examples.

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Response: often between 1 and 5 words

---

Context: The Scholastic Magazine of Notre Dame began publishing in September 1876.

Question: When did the Scholastic Magazine of Notre dame begin publishing?

Reasoning: Let's think step by step in order to produce the answer. We know that the Scholastic Magazine of Notre Dame began publishing in September 1876.

Response: September 1876

---

Context: The daily student paper at Notre Dame is called "The Observer."

Question: What is the daily student paper at Notre Dame called?

Reasoning: Let's think step by step in order to produce the answer. We know that Notre Dam

In [8]:
import json
import tqdm

def create_bakeoff_submission(model, file_name="cs224u-openqa-bakeoff-entry.json"):
    """"
    The argument `model` is a `dspy.Module`. The return value of its
    `forward` method must have an `answer` attribute.
    """

    filename = os.path.join("data", "openqa", "cs224u-openqa-test-unlabeled.txt")

    # This should become a mapping from questions (str) to response
    # dicts from your system.
    gens = {}

    with open(filename) as f:
        questions = f.read().splitlines()

    # Here we loop over the questions, run the system `model`, and
    # store its `answer` value as the prediction:
    for question in tqdm.tqdm(questions):
        try:
            gens[question] = model(question=question).answer
        except:
            print("Didn't work for the following prompt {}".format(question))
            gens[question] = ""
            continue

    # Quick tests we advise you to run:
    # 1. Make sure `gens` is a dict with the questions as the keys:
    assert all(question in gens for q in questions)
    # 2. Make sure the values are str:
    assert all(isinstance(d, str) for d in gens.values())

    # And finally the output file:
    with open(file_name, "wt") as f:
        json.dump(gens, f, indent=4)

# create_bakeoff_submission(model, file_name="cs224u-openqa-bakeoff-entry-base-model_original_index.json")
create_bakeoff_submission(bootstrap_model, file_name="cs224u-openqa-bakeoff-entry-bootstrapmodel_original_index.json")
# create_bakeoff_submission(bootstrap_random_search_model, file_name="cs224u-openqa-bakeoff-entry-bootstrapwithrandonsearch_original_index.json")

  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/400 [00:00<?, ?it/s]1

In [4]:
#chain of thought with few shot 
tiny_evaluater(model, metric=answer_exact_match)

Average Metric: 0 / 15  (0.0): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 346.82it/s]


Average Metric: 0 / 15  (0.0%)


,question,example_answer,context,rationale,pred_answer,answer_exact_match
0,In 1517 who was Luther's bishop?,Albert of Mainz,"['Johann V Thurzo | John V Thurzó (Hungarian: ""Thurzó V. János"" ; Polish: ""Jan V Turzo"" ) was a 15th-century Bishop of Wrocław, Poland. A...",produce the answer. We know that Luther's bishop in 1517 was Thomas Cajetan.,Thomas Cajetan,False
1,When was the construction that changed the Rhine's Delta?,20th Century,"['Nieuwe Merwede | The Nieuwe Merwede (""New Merwede"") is a canal, mainly fed by the river Rhine, that was constructed in 1870 to form a...",produce the answer. We know that the Nieuwe Merwede canal was constructed in 1870 to form a branch in the Rhine-Meuse delta. This construction aimed...,1870,False
2,How many companies were registered in Warsaw in 2006?,"304,016","[""WIG | WIG, originally an acronym for Warszawski Indeks Giełdowy (Warsaw Stock Exchange Index) is the oldest index of the Warsaw Stock Exchange, introduced on...","produce the answer. We know that the WIG index lists 318 companies as of 2010. However, we need to find the number of companies registered...",unknown,False
3,What is the CJEU's duty?,"to ""ensure that in the interpretation and application of the Treaties the law is observed""",['Combined Joint Expeditionary Force | The Combined Joint Expeditionary Force (CJEF) is a Franco-British military force. It draws upon both the British Armed Forces and...,produce the answer. We know that the CJEU is the institution of the European Union that encompasses the whole judiciary. It consists of two separate...,to encompass the whole judiciary,False
4,What would a teacher do for someone who is cocky?,deflate,"[""Principal teacher | A Principal Teacher (PT), in Scotland is a promoted post within a school. In primary schools, a Principal Teacher can either be...","produce the answer. We know that a teacher helps others to acquire knowledge, competences, or values. If someone is cocky, a teacher might help them...",Teach them humility,False


0.0

In [4]:
# bootstrap few shot optimizer
tiny_evaluater(model, metric=answer_exact_match)

Average Metric: 1 / 15  (6.7): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:03<00:00,  3.77it/s]


Average Metric: 1 / 15  (6.7%)


,question,example_answer,context,pred_answer,answer_exact_match
0,In 1517 who was Luther's bishop?,Albert of Mainz,"['Johann V Thurzo | John V Thurzó (Hungarian: ""Thurzó V. János"" ; Polish: ""Jan V Turzo"" ) was a 15th-century Bishop of Wrocław, Poland. A...",John V Thurzó,False
1,When was the construction that changed the Rhine's Delta?,20th Century,"['Nieuwe Merwede | The Nieuwe Merwede (""New Merwede"") is a canal, mainly fed by the river Rhine, that was constructed in 1870 to form a...",1870,False
2,How many companies were registered in Warsaw in 2006?,"304,016","[""WIG | WIG, originally an acronym for Warszawski Indeks Giełdowy (Warsaw Stock Exchange Index) is the oldest index of the Warsaw Stock Exchange, introduced on...",318,False
3,What is the CJEU's duty?,"to ""ensure that in the interpretation and application of the Treaties the law is observed""",['Combined Joint Expeditionary Force | The Combined Joint Expeditionary Force (CJEF) is a Franco-British military force. It draws upon both the British Armed Forces and...,To encompass the whole judiciary of the European Union.,False
4,What would a teacher do for someone who is cocky?,deflate,"[""Principal teacher | A Principal Teacher (PT), in Scotland is a promoted post within a school. In primary schools, a Principal Teacher can either be...",help them acquire humility,False


6.67

In [35]:
# Chain of thought
tiny_evaluater(model, metric=answer_exact_match)

Average Metric: 2 / 15  (13.3): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 671.88it/s]

Average Metric: 2 / 15  (13.3%)



/anaconda/envs/cs224u-cpu/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


,question,example_answer,context,pred_answer,answer_exact_match
0,In 1517 who was Luther's bishop?,Albert of Mainz,"[""Paul Speratus | Ellwangen, Priest of the Diocese of Augsburg). Early studies took him to Paris and Italy, as well as (probably) Freiburg and Vienna....",Paul Speratus,False
1,When was the construction that changed the Rhine's Delta?,20th Century,"['Rhine | rivers and streams. Many rivers have been closed (""dammed"") and now serve as drainage channels for the numerous polders. The construction of Delta...",Second half of the 20th Century,False
2,How many companies were registered in Warsaw in 2006?,"304,016","['Warsaw | such as Sydney, Istanbul, Amsterdam or Seoul. Warsaw, especially its city centre (""Śródmieście""), is home not only to many national institutions and government...","304,016",✔️ [True]
3,What is the CJEU's duty?,"to ""ensure that in the interpretation and application of the Treaties the law is observed""",['European Union law | elected by the judges for three years. While TEU article 19(3) says the Court of Justice is the ultimate court to...,Ensure law is observed.,False
4,What would a teacher do for someone who is cocky?,deflate,"['Teacher | for the individual students accordingly. For example, an experienced teacher and parent described the place of a teacher in learning as follows: ""The...",Deflate the cocky.,False


13.33

In [29]:
# without summarization just OpenQA with retriever
tiny_evaluater(model, metric=answer_exact_match)

Average Metric: 3 / 15  (20.0): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 653.48it/s]

Average Metric: 3 / 15  (20.0%)



/anaconda/envs/cs224u-cpu/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


,question,example_answer,pred_answer,answer_exact_match
0,In 1517 who was Luther's bishop?,Albert of Mainz,Paul Speratus,False
1,When was the construction that changed the Rhine's Delta?,20th Century,second half of the 20th Century,False
2,How many companies were registered in Warsaw in 2006?,"304,016","304,016",✔️ [True]
3,What is the CJEU's duty?,"to ""ensure that in the interpretation and application of the Treaties the law is observed""","""ensure that in the interpretation and application of the Treaties the law is observed""",False
4,What would a teacher do for someone who is cocky?,deflate,deflate,✔️ [True]


20.0

In [17]:
# baseline: Summarize QA  given by assignment
tiny_evaluater(model, metric=answer_exact_match)

Average Metric: 1 / 15  (6.7): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 317.75it/s]

Average Metric: 1 / 15  (6.7%)


,question,example_answer,context,pred_answer,answer_exact_match
0,In 1517 who was Luther's bishop?,Albert of Mainz,"The given passages provide brief biographical information about three individuals: Paul Speratus, George of the Palatinate, and Georg Schenk von Limpurg. Paul Speratus was a...",George of the Palatinate,False
1,When was the construction that changed the Rhine's Delta?,20th Century,The passages discuss the Rhine-Meuse Delta and the construction of the Delta Works in the Netherlands. The Delta Works were built to prevent flooding in...,second half of the 20th century,False
2,How many companies were registered in Warsaw in 2006?,"304,016","Warsaw is a city with a strong economy and a growing business community. It is home to many national institutions, government agencies, and domestic and...",Not provided.,False
3,What is the CJEU's duty?,"to ""ensure that in the interpretation and application of the Treaties the law is observed""",The context provided discusses the role and function of the Court of Justice of the European Union (CJEU) in interpreting and applying European Union (EU)...,To ensure that the law is observed.,False
4,What would a teacher do for someone who is cocky?,deflate,"The passages provided discuss different aspects of the concept of being cocky. The first passage describes the role of a teacher in learning, highlighting that...",motivate and guide,False


6.67

In [18]:
lm.inspect_history(n=1)





Answer questions with short factoid answers.

---

Follow the following format.

Context: may contain relevant facts
Question: ${question}
Answer: often between 1 and 5 words

---

Context: The context discusses the different categories of schools in South Africa, including state schools and independent schools. State schools are state-owned and can have a degree of budget autonomy, while independent schools include private schools and schools that are privately governed. The private school sector in South Africa has grown significantly since the abolition of apartheid, with an increase in the number of registered independent schools. Traditional, high-fee independent schools are now a minority in the sector, with the majority of schools charging lower fees.
Question: In South Africa, along with privately governed schools, what schools are classified as independent?
Answer: Private schools





In [9]:
from dspy.retrieve.you_rm import YouRM 

In [10]:
yourm = YouRM(k=5)

RuntimeError: You must supply ydc_api_key or set environment variable YDC_API_KEY